In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as Pickle
import spacy
import random

from fastai.structured import *
from fastai.column_data import *

In [2]:
bs,bptt = 64,70

## Language modeling

### Data

In [3]:
import pandas as pd

In [4]:
PATH='data/high-flyers/'

df_tweets = pd.read_csv(f'{PATH}train.csv', index_col=0)
df_test = pd.read_csv(f'{PATH}test.csv', index_col=0)

In [5]:
from IPython.display import HTML, display
display(df_tweets.head())

,text,airline,tweet_location,user_timezone,sentiment
id,,,,,
0,@JetBlue great flight! Great view! :-) http://...,Delta,NaN,NaN,positive
1,"@united they're not, actually. gate agent was ...",United,chicago,NaN,negative
2,@AmericanAir No worries they called back 4 hrs...,American,"Dallas, Texas",NaN,negative
3,@united thank you. There was one here a few mo...,United,"New York, NY",America/New_York,positive
4,@united Brothers luggage was lost on Copa Airl...,United,"Kearney, Nebraska",Central Time (US & Canada),negative


In [6]:
display(DataFrameSummary(df_tweets).summary())

,text,airline,tweet_location,user_timezone,sentiment
count,10387,10387,6972,6901,10387
unique,10258,6,2429,76,2
top,@AmericanAir thanks,United,"Washington, DC",Eastern Time (US & Canada),negative
freq,5,2822,120,2673,8242
counts,10387,10387,6972,6901,10387
uniques,10258,6,2429,76,2
missing,0,0,3415,3486,0
missing_perc,0%,0%,32.88%,33.56%,0%
types,categorical,categorical,categorical,categorical,bool


In [7]:
df_tweets = df_tweets.drop(columns=["airline", "tweet_location", "user_timezone"])

In [8]:
my_tok = spacy.load('en_core_web_lg')
# TEXT = data.Field(lower=True, tokenize=my_tok)
TEXT = data.Field(lower=True, tokenize="spacy")

In [9]:
os.makedirs(f'{PATH}trn/positive', exist_ok=True)
os.makedirs(f'{PATH}val/positive', exist_ok=True)
os.makedirs(f'{PATH}trn/negative', exist_ok=True)
os.makedirs(f'{PATH}val/negative', exist_ok=True)
os.makedirs(f'{PATH}all/trn', exist_ok=True)
os.makedirs(f'{PATH}all/val', exist_ok=True)
os.makedirs(f'{PATH}models', exist_ok=True)

In [10]:
val_idxs = get_cv_idxs(len(df_tweets))
df_val = df_tweets.loc[val_idxs]
df_val.head()

,text,sentiment
id,,
8954,"@USAirways figure it out, have had 2 Cancelled...",negative
6495,"@AmericanAir checked in at Des Moines, lay ove...",negative
4184,@united more people would catch their flights ...,negative
7015,"@AmericanAir I can't!!! No one can't get to ""...",negative
5860,@USAirways Now I am probably going to miss my ...,negative


In [11]:
df_trn = df_tweets.drop(df_tweets.index[val_idxs])
df_trn[:25]

,text,sentiment
id,,
1,"@united they're not, actually. gate agent was ...",negative
2,@AmericanAir No worries they called back 4 hrs...,negative
4,@united Brothers luggage was lost on Copa Airl...,negative
5,"@AmericanAir Great, thanks. Followed.",positive
6,@USAirways You all work hard at making sure th...,positive
7,@united we have been told that it's lost...you...,negative
9,"@united we got it, thanks.",positive
11,@AmericanAir ahhhh your silence is golden now....,negative
12,@SouthwestAir looks like you are up and runnin...,negative


In [12]:
md = LanguageModelData.from_dataframes(f'{PATH}all/', TEXT, "text",df_trn, df_val,test_df=df_test, bs=bs, bptt=bptt, min_freq=5)
# LanguageModelData.from_text_files(f'{PATH}all/', TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

In [13]:
os.makedirs(f'{PATH}models', exist_ok=True)
Pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [14]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(45, 2229, 1, 206924)

In [15]:
TEXT.vocab.itos[:12]

['<unk>', '<pad>', '.', '<', '>', 'eos', 'to', 'i', 'the', '!', 'you', 'a']

In [16]:
' '.join(md.trn_ds[0].text[:150])

"@united they 're not , actually . gate agent was so rude . now standing in a line waiting for reflight booking problems . missed the only flight to sti . awful . < eos > @americanair no worries they called back 4 hrs late flightr while i was asleep and took an additional $ 200 fee . so by aa standards everything 's gr8 < eos > @united brothers luggage was lost on copa airlines flight 635 . he 's competing sunday for the 2015 panamerican cross country cup . please help < eos > @americanair great , thanks . followed . < eos > @usairways you all work hard at making sure things flow smoothly . keeping positive . < eos > @united we have been told that it 's lost ... you guys do n't know where it is < eos > @united we got it ,"

### Train

In [24]:
em_sz = 200
nh = 500
nl = 3
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [25]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
    dropout=0.05, dropouth=0.1, dropouti=0.05, dropoute=0.02, wdrop=0.2)
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
#                dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [26]:
learner.fit(3e-3, 1, wds=1e-6)

epoch      trn_loss   val_loss                                                                                         
    0      5.449676   4.958768  



[array([4.95877])]

In [27]:
learner.fit(3e-3, 3, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss                                                                                         
    0      4.924763   4.806131  
    1      4.799543   4.60139                                                                                          
    2      4.675151   4.544723                                                                                         
    3      4.570612   4.347907                                                                                         
    4      4.420116   4.227324                                                                                         
    5      4.295838   4.177052                                                                                         
    6      4.223622   4.164743                                                                                         



[array([4.16474])]

In [28]:
learner.save_encoder('adam2_enc')

In [29]:
learner.fit(3e-3, 2, wds=1e-6, cycle_len=5, cycle_save_name='adam3_10')

epoch      trn_loss   val_loss                                                                                         
    0      4.182064   4.120918  
    1      4.116482   4.033933                                                                                         
    2      4.039857   3.988093                                                                                         
    3      3.979326   3.963594                                                                                         
    4      3.933965   3.960349                                                                                         
    5      3.945585   3.946581                                                                                         
    6      3.890913   3.891517                                                                                         
    7      3.820585   3.868454                                                                                         
    8  

[array([3.84607])]

In [30]:
learner.save_encoder('adam3_10_enc')

### Test

In [31]:
def proc_str(s): return TEXT.preprocess(TEXT.tokenize(s))
def num_str(s): return TEXT.numericalize([proc_str(s)])

In [32]:
m=learner.model

In [33]:
s="""service that is"""

In [34]:
def sample_model(m, s, l=50):
    t = num_str(s)
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(t)
    print('...', end='')

    for i in range(l):
        n=res[-1].topk(2)[1]
        n = n[1] if n.data[0]==0 else n[0]
        word = TEXT.vocab.itos[n.data[0]]
        print(word, end=' ')
        if word=='<eos>': break
        res,*_ = m(n[0].unsqueeze(0))

    m[0].bs=bs

In [35]:
sample_model(m,"flight was")

...cancelled flightled . < eos > @united i 'm not sure it 's not a good airline . < eos > @united i 'm not sure it 's not a good airline . < eos > @united i 'm not sure it 's not a good airline . < eos 

In [36]:
sample_model(m,"people are")

...n't even even . < eos > @united i 'm not sure it 's not a good airline . < eos > @united i 'm not sure it 's not a good airline . < eos > @united i 'm not sure it 's not a good airline . < 

In [37]:
sample_model(m,"food was")

...n't a worst . < eos > @united i 'm not sure it 's not a good airline . < eos > @united i 'm not sure it 's not a good airline . < eos > @united i 'm not sure it 's not a good airline . < 

In [38]:
sample_model(m,"ride was ")

...a worst airline . < eos > @united i 'm not sure it 's not a good airline . < eos > @united i 'm not sure it 's not a good airline . < eos > @united i 'm not sure it 's not a good airline . < 

In [39]:
sample_model(m,"checkin was ")

...a cancelled flightled flight . < eos > @united i 'm not sure it 's not a good airline . < eos > @united i 'm not sure it 's not a good airline . < eos > @united i 'm not sure it 's not a good airline . 

In [40]:
sample_model(m,"check in was ")

...a worst . < eos > @united i 'm not sure it 's not a good airline . < eos > @united i 'm not sure it 's not a good airline . < eos > @united i 'm not sure it 's not a good airline . < eos 

### Sentiment

In [41]:
TEXT = Pickle.load(open(f'{PATH}models/TEXT.pkl','rb'))

In [42]:
from torchtext import data

class DataFrameDataset(data.Dataset):

    def __init__(self, df, text_field, label_field, is_test=False, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        examples = []
        for i, row in df.iterrows():
            label = row.sentiment if not is_test else None
            text = row.text
            examples.append(data.Example.fromlist([text, label], fields))

        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex):
        return len(ex.text)

    @classmethod
    def splits(cls, text_field, label_field, train_df, val_df=None, test_df=None, **kwargs):
        train_data, val_data, test_data = (None, None, None)

        if train_df is not None:
            train_data = cls(train_df.copy(), text_field, label_field, **kwargs)
        if val_df is not None:
            val_data = cls(val_df.copy(), text_field, label_field, **kwargs)
        if test_df is not None:
            test_data = cls(test_df.copy(), text_field, label_field, True, **kwargs)

        return tuple(d for d in (train_data, val_data, test_data) if d is not None)
        

In [43]:
TWEET_LABEL = data.Field(sequential=False)
splits = DataFrameDataset.splits(TEXT, TWEET_LABEL, train_df=df_trn, val_df=df_val, test_df=df_test)

#train_ds, val_ds, test_ds = DataFrameDataset.splits(
#  text_field=TEXT_FIELD, label_field=LABEL_FIELD, train_df=train_df, val_df=val_df, test_df=test_df)

In [44]:
md2 = TextData.from_splits(PATH, splits, bs)

In [45]:
#            dropout=0.3, dropouti=0.4, wdrop=0.3, dropoute=0.05, dropouth=0.2)

In [46]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

def prec_at_6(preds,targs):
    precision, recall, _ = precision_recall_curve(targs==2, preds[:,2])
    print(recall[precision>=0.6][0])
    return recall[precision>=0.6][0]

In [47]:
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.65, wdrop=0.5, dropoute=0.1, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.clip=25.

In [48]:
# this notebook has a mess of some things going under 'all/' others not, so a little hack here
# !ln -sf ../all/models/adam3_20_enc.h5 {PATH}models/adam3_20_enc.h5
# m3.load_encoder(f'adam3_20_enc')
lrs=np.array([1e-4,1e-3,1e-3,1e-2,3e-2])

In [49]:
m3.freeze_to(-1)
m3.fit(lrs/2, 10, metrics=[accuracy])
m3.unfreeze()
m3.fit(lrs, 10, metrics=[accuracy], cycle_len=1)

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.56917    0.596086   0.848165  
    1      0.546411   0.4562     0.846302                                                                              
    2      0.525474   0.450245   0.846302                                                                              
    3      0.519524   0.466743   0.846302                                                                              
    4      0.512077   0.500114   0.846302                                                                              
    5      0.527203   0.45717    0.846302                                                                              
    6      0.519083   0.481592   0.846302                                                                              
    7      0.516466   0.440719   0.846302                                                                           

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.439265   0.352422   0.854373  
    1      0.402351   0.343455   0.858533                                                                              
    2      0.362576   0.298844   0.874924                                                                              
    3      0.334206   0.272644   0.88732                                                                               
    4      0.341494   0.27917    0.885437                                                                              
    5      0.300068   0.256837   0.898475                                                                              
    6      0.305594   0.250445   0.90487                                                                               
    7      0.288786   0.248462   0.903256                                                                           

[array([0.2199]), 0.9171072175171202]

In [50]:
m3.fit(lrs, 2, metrics=[accuracy], cycle_len=4, cycle_save_name='imdb2')

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.263689   0.354947   0.854545  
    1      0.273172   0.349382   0.907643                                                                              
    2      0.253552   0.203729   0.918204                                                                              
    3      0.250235   0.204578   0.919611                                                                              
    4      0.335645   0.290875   0.896453                                                                              
    5      0.267692   0.201408   0.923516                                                                              
    6      0.260756   0.200282   0.920853                                                                              
    7      0.232334   0.205634   0.920453                                                                           

[array([0.20563]), 0.9204534415377686]

In [51]:
prec_at_6(*m3.predict_with_targs())

0.8701923076923077


0.8701923076923077

In [52]:
m3.fit(lrs, 4, metrics=[accuracy], cycle_len=2, cycle_save_name='imdb2')

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.245747   0.234727   0.910354  
    1      0.224843   0.199423   0.922978                                                                              
    2      0.237376   0.193804   0.928359                                                                              
    3      0.223704   0.191388   0.927014                                                                              
    4      0.258551   0.217588   0.919915                                                                              
    5      0.210141   0.20183    0.927262                                                                              
    6      0.228848   0.210388   0.923641                                                                              
    7      0.207218   0.183593   0.934485                                                                           

[array([0.18359]), 0.934484898491411]

In [53]:
prec_at_6(*m3.predict_with_targs())

0.8701923076923077


0.8701923076923077

In [54]:
m3.fit(lrs, 4, metrics=[accuracy], cycle_len=2, cycle_save_name='imdb2')

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.205307   0.189635   0.926627  
    1      0.207246   0.18413    0.93107                                                                               
    2      0.210439   0.190705   0.929766                                                                              
    3      0.19805    0.176028   0.938376                                                                              
    4      0.208255   0.601144   0.913562                                                                              
    5      0.188756   0.180265   0.938334                                                                              
    6      0.219985   0.269673   0.912693                                                                              
    7      0.23687    0.213948   0.924413                                                                           

[array([0.21395]), 0.9244126810932791]

In [55]:
m3.fit(lrs, 4, metrics=[accuracy], cycle_len=2, cycle_save_name='imdb2')

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.216619   0.184368   0.932043  
    1      0.191975   0.181562   0.933905                                                                              
    2      0.202262   0.289772   0.925338                                                                              
    3      0.181586   0.282176   0.929435                                                                              
    4      0.213935   0.799902   0.911679                                                                              
    5      0.178181   0.364243   0.924924                                                                              
    6      0.20438    0.183556   0.932664                                                                              
    7      0.169562   0.17612    0.933512                                                                           

[array([0.17612]), 0.9335122163722057]

In [56]:

# rnn_model = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
#           dropout=0.1, dropouti=0.65, wdrop=0.5, dropoute=0.1, dropouth=0.3)

# ...

# rnn_model.data.test_dl.src.sort = False
# rnn_model.data.test_dl.src.sort_within_batch = False
# rnn_model.data.test_dl.src.shuffle = False

probs = m3.predict(is_test=True)
preds = np.argmax(probs, axis=1)

pd.DataFrame({
    'id': df_test.index,
    'sentiment': [TWEET_LABEL.vocab.itos[p] for p in preds]}).to_csv('data/high-flyers/sub6.csv', index=False)
    
FileLink('data/high-flyers/sub6.csv')

C:\Users\nissan.dookeran\Documents\Github\fastai\courses\dl1\data\high-flyers\sub6.csv